In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,302.14,84,100,3.09,CC,1706842090
1,1,port-aux-francais,-49.3500,70.2167,280.52,88,16,8.16,TF,1706841809
2,2,badger,64.8000,-147.5333,238.60,69,75,0.00,US,1706842091
3,3,adamstown,-25.0660,-130.1015,298.00,88,53,8.76,PN,1706842092
4,4,nuuk,64.1835,-51.7216,258.49,85,100,4.63,GL,1706842092


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
map_plot_2 = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    label="City",
    color="City",
    cmap="Category10"
)

map_plot_2

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.City :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[(city_data_df["Max Temp"] < 300)]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()


# Display sample data
filtered_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,port-aux-francais,-49.3500,70.2167,280.52,88,16,8.16,TF,1706841809
2,2,badger,64.8000,-147.5333,238.60,69,75,0.00,US,1706842091
3,3,adamstown,-25.0660,-130.1015,298.00,88,53,8.76,PN,1706842092
4,4,nuuk,64.1835,-51.7216,258.49,85,100,4.63,GL,1706842092
5,5,bethel,41.3712,-73.4140,278.67,78,100,1.31,US,1706842092


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df= hotel_df[["City","Country","Lat","Lng", "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,west island,CC,-12.1568,96.8225,84,
1,port-aux-francais,TF,-49.3500,70.2167,88,
2,badger,US,64.8000,-147.5333,69,
3,adamstown,PN,-25.0660,-130.1015,88,
4,nuuk,GL,64.1835,-51.7216,85,
...,...,...,...,...,...,...
563,mastic beach,US,40.7668,-72.8521,91,
564,cockburn town,TC,21.4612,-71.1419,58,
565,madang,PG,-5.2246,145.7966,74,
566,moa,CU,20.6569,-74.9403,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng= row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
west island - nearest hotel: Cocos Village Bungalows
port-aux-francais - nearest hotel: Keravel
badger - nearest hotel: Hotel North Pole
adamstown - nearest hotel: No hotel found
nuuk - nearest hotel: Nordbo
bethel - nearest hotel: Hampton Inn Danbury
waitangi - nearest hotel: Hotel Chathams
kapuskasing - nearest hotel: Travelodge
manta - nearest hotel: Hotel Santa Fe
warrnambool - nearest hotel: Central Court
edinburgh of the seven seas - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
blackmans bay - nearest hotel: Villa Howden
arish - nearest hotel: سينا
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
qaqortoq - nearest hotel: Hotel Qaqortoq
lerwick - nearest hotel: Grand Hotel
puerto natales - nearest hotel: Dorotea Patagonia Hostel
naze - nearest hotel: ビジネス旅館畠山
whakatane - nearest hotel: Whakatane Hotel
tolanaro - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
0,west island,CC,-12.1568,96.8225,84,Cocos Village Bungalows
1,port-aux-francais,TF,-49.3500,70.2167,88,Keravel
2,badger,US,64.8000,-147.5333,69,Hotel North Pole
3,adamstown,PN,-25.0660,-130.1015,88,No hotel found
4,nuuk,GL,64.1835,-51.7216,85,Nordbo
...,...,...,...,...,...,...
563,mastic beach,US,40.7668,-72.8521,91,Temperance Hall
564,cockburn town,TC,21.4612,-71.1419,58,The Salt Raker Inn
565,madang,PG,-5.2246,145.7966,74,Madang Star International
566,moa,CU,20.6569,-74.9403,66,Hotel Miraflores


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.


In [20]:
# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    label="City",
    color="City",
    cmap="Category10",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_3

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.City :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)